In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [5]:
spark=SparkSession.builder.appName('Web Log Report Analysis').enableHiveSupport().getOrCreate()

In [6]:
columns=['Timestamp','ReportType','Target','Referrer','Link','SessionId','SessionCount','PageTitle','LoadTime','ViewTime','Embedded','Cookie','HSResponseTime','PrefetchElement'
,'ElementsinHints','HintAlreadySeen','Viewedfor1sttimePrefetched','Viewed1sttimenotPrefetched','ConxSpeed','ConxType','PrevConxType','VisitstoOrder','DaystoOrder','VisitFreq'
,'PurchaseFreq','VisChip','TimeinSession','PreprocRules','Secondssincelastpage','ScreenResolution','ColorDepth','CookiesEnabled','ReferringURL','Product1stVisit','FlashVersion'
,'UserAgent','RemoteIP','Serial','TargetMatches','NormalizedTarget','ThirdPartyCookieEnabled','Dummy']            

In [7]:
df =spark.read.option("delimiter", "}")\
            .csv(r"D:\Projects\Udemy\web_log_report_analysis\Data.txt",inferSchema='true')\
            .toDF(*columns)\
            .withColumn('SessionCount',col('SessionCount').cast('bigint'))\
            .drop(col('Dummy'))

In [8]:
df.createOrReplaceTempView('weblog')

In [9]:
spark.sql('select count(*) as count from weblog').show()

+-----+
|count|
+-----+
|  219|
+-----+



### Session Report
#### The number of sessions within the selected time frame

#### Session Count - Session count is a record of the number of times a user visits your website within a given timeframe, such as day, week, or month. A cookie on the visitor’s browser is used to track the user’s activity between the time they enter your domain and the time they leave.

In [10]:
spark.sql("""select from_unixtime(Timestamp, "yyyy-MM-dd") as Date
                ,from_unixtime(Timestamp, "HH") as Time, sum(SessionCount) as SessionCountByHour
                ,(sum(SessionCount)/(select sum(SessionCount) from weblog) * 100) as Percentage  
            from weblog 
            group by Timestamp order by Time""").show()

+----------+----+------------------+------------------+
|      Date|Time|SessionCountByHour|        Percentage|
+----------+----+------------------+------------------+
|2019-05-18|  07|                84| 4.017216642754662|
|2019-05-18|  08|               177| 8.464849354375897|
|2019-05-18|  09|               286| 13.67766618842659|
|2019-05-18|  09|               231|11.047345767575322|
|2019-05-18|  10|               259|12.386417981826877|
|2019-05-18|  11|               461|22.046867527498804|
|2019-05-18|  12|               229| 10.95169775227164|
|2019-05-18|  13|               364|17.407938785270204|
+----------+----+------------------+------------------+



### Page Views Report
#### The number of page views within the selected time frame.

#### Page Views - A pageview is each time a visitor views a page on your website, regardless of how many hits are generated

In [11]:
spark.sql("""
select from_unixtime(Timestamp, "yyyy-MM-dd") as Date, 
        from_unixtime(Timestamp, "HH") as Time, 
        count(PageTitle) as PagesViews, 
        (count(PageTitle)/(select count(*) from weblog) * 100) as Percentage 
from  weblog group by Timestamp order by Time""").show()

+----------+----+----------+------------------+
|      Date|Time|PagesViews|        Percentage|
+----------+----+----------+------------------+
|2019-05-18|  07|        11|5.0228310502283104|
|2019-05-18|  08|        15|6.8493150684931505|
|2019-05-18|  09|        30|13.698630136986301|
|2019-05-18|  09|        28|12.785388127853881|
|2019-05-18|  10|        32| 14.61187214611872|
|2019-05-18|  11|        47|21.461187214611872|
|2019-05-18|  12|        22|10.045662100456621|
|2019-05-18|  13|        34| 15.52511415525114|
+----------+----+----------+------------------+



### New Visitor Report
#### New Visitors are the number of distinct New users that have visited the Website during a given time period.
###### _Using Dataframe Operations_

In [12]:

df.select(from_unixtime('Timestamp', "yyyy-MM-dd").alias('Date'),\
            from_unixtime('Timestamp', "HH").alias('Time'),'Viewedfor1sttimePrefetched')\
            .filter(col('Viewedfor1sttimePrefetched')=='YES')\
            .groupBy(col('Date'),col('Time')).agg(count('*').alias('New_Visitor')).orderBy(col('Time')).show()

+----------+----+-----------+
|      Date|Time|New_Visitor|
+----------+----+-----------+
|2019-05-18|  07|          6|
|2019-05-18|  08|          8|
|2019-05-18|  09|         28|
|2019-05-18|  10|         16|
|2019-05-18|  11|         30|
|2019-05-18|  12|          8|
|2019-05-18|  13|         19|
+----------+----+-----------+



#### Using SQL Operations

In [13]:
spark.sql("""select Referrer, from_unixtime(Timestamp, "yyyy-MM-dd") as Date, 
                from_unixtime(Timestamp, "HH") as Time, 
                count(Viewedfor1sttimePrefetched) as NewVisitor 
            from weblog 
                where Viewedfor1sttimePrefetched = "YES" 
            group by referrer ,Timestamp order by count(Viewedfor1sttimePrefetched) desc""").show()

+-------------------+----------+----+----------+
|           Referrer|      Date|Time|NewVisitor|
+-------------------+----------+----+----------+
|   www.snapchat.com|2019-05-18|  11|         3|
|      www.skype.com|2019-05-18|  09|         3|
|     www.wechat.com|2019-05-18|  11|         3|
|www.tieba.baidu.com|2019-05-18|  11|         3|
|      www.weibo.com|2019-05-18|  09|         2|
|     www.reddit.com|2019-05-18|  11|         2|
|   www.qzone.qq.com|2019-05-18|  08|         2|
|      www.viber.com|2019-05-18|  13|         2|
|  www.instagram.com|2019-05-18|  08|         2|
|     www.google.com|2019-05-18|  11|         2|
|      www.viber.com|2019-05-18|  09|         2|
|         www.vk.com|2019-05-18|  09|         2|
|   www.whatsapp.com|2019-05-18|  09|         2|
|        www.line.me|2019-05-18|  09|         2|
|      www.weibo.com|2019-05-18|  11|         2|
|    www.taringa.net|2019-05-18|  11|         2|
|      www.viber.com|2019-05-18|  11|         2|
|      www.skype.com

### Referring Domains Report
#### Referring domains are Web sites that end users visited before going to your Web site. They can indicate popular link to your Website.

In [14]:
spark.sql("""SELECT Referrer, sum(SessionCount) as Session, count(Referrer) as Orders, 
                    sum(split(NormalizedTarget, '/')[1]) as `Revenue(in $)` from weblog group by Referrer;""").show()

+-------------------+-------+------+-------------+
|           Referrer|Session|Orders|Revenue(in $)|
+-------------------+-------+------+-------------+
|    www.taringa.net|    105|    10|      47546.0|
|   www.snapchat.com|     96|    12|      62749.0|
|         www.qq.com|    130|    12|      55367.0|
| www.foursquare.com|     83|     8|      48707.0|
|www.tieba.baidu.com|     57|     6|      33008.0|
|     www.renren.com|     89|    13|      58873.0|
|     www.tumblr.com|     80|     9|      33580.0|
|     www.reddit.com|    118|    13|      74768.0|
|     www.wechat.com|     93|     8|      22479.0|
|     www.google.com|    105|    10|      44768.0|
|   www.facebook.com|     87|     9|      43438.0|
|        www.line.me|     84|    11|      57456.0|
|   www.telegram.org|     70|     7|      35249.0|
|      www.weibo.com|     80|     8|      51534.0|
|    www.twitter.com|     64|     5|      17148.0|
|      www.skype.com|     93|    10|      37012.0|
|         www.vk.com|    103|  

### Top IP Addresses Report
#### This report ranks the IP addresses of visitors accessing your website in terms of number of sessions

In [15]:
spark.sql("""SELECT RemoteIP, sum(SessionCount) as Session, count(RemoteIP) as Orders, 
sum(split(NormalizedTarget, '/')[1]) as Revenue 
from weblog group by RemoteIP order by sum(SessionCount) desc""").show()

+-------------+-------+------+-------+
|     RemoteIP|Session|Orders|Revenue|
+-------------+-------+------+-------+
|     10.0.1.1|     95|    10|53226.0|
| 192.168.1.10|     86|     9|52344.0|
|  192.168.1.1|     86|    10|58979.0|
| 192.168.0.10|     78|     8|45170.0|
| 192.168.1.99|     77|     6|24334.0|
| 192.168.55.1|     74|     7|26139.0|
|192.168.10.50|     70|     6|33529.0|
| 192.168.11.1|     68|     7|30694.0|
|     10.0.0.2|     64|     5|20707.0|
|192.168.1.100|     62|     6|22358.0|
|192.168.10.10|     60|     5|27378.0|
|    10.1.10.1|     58|     6|33138.0|
|  192.168.8.1|     57|     6|36657.0|
|  192.168.3.1|     54|     7|44057.0|
|  192.168.0.1|     52|     5|16627.0|
|192.168.0.254|     48|     6|35383.0|
| 192.168.15.1|     47|     4| 7384.0|
|   10.0.0.138|     47|     4|12454.0|
|192.168.2.254|     45|     4|21894.0|
|  192.168.4.1|     44|     5|23283.0|
+-------------+-------+------+-------+
only showing top 20 rows



###  Search Query Report
* Search queries are the key words entered into Internet search engines that provided results directing end users to your Web site.
* This report depicts the top search queries that led users to your site and allows you to compare the number of page hits received by each search query

In [16]:
df.select(split('PreprocRules','=')[1].alias('SearchQuery'))\
            .groupBy(col('SearchQuery')).agg(count('*').alias('Requests'))\
            .orderBy(desc('Requests')).show(truncate=False)


+----------------------+--------+
|SearchQuery           |Requests|
+----------------------+--------+
|Mobile                |15      |
|Health Care Appliances|14      |
|Television            |14      |
|Desktop PCs           |13      |
|Watches               |13      |
|Kitchen Appliances    |12      |
|Beauty and Grooming   |12      |
|Home Entertainment    |11      |
|Laptops               |11      |
|Mens Footware         |10      |
|Computer Accessories  |9       |
|Refrigerators         |9       |
|Accessories           |8       |
|Womens Footware       |7       |
|Washing Machines      |7       |
|Mens Clothing         |7       |
|Table                 |7       |
|Android TV            |7       |
|Mobile Accessories    |6       |
|Womens Clothing       |6       |
+----------------------+--------+
only showing top 20 rows



#### Browser Used for Shopping

In [17]:
spark.sql('select UserAgent as Browser, count(UserAgent)  from weblog group by UserAgent order by count(UserAgent) desc').show()

+-----------------+----------------+
|          Browser|count(UserAgent)|
+-----------------+----------------+
|        Pale Moon|              24|
|         Basilisk|              18|
|           Chrome|              18|
|           Safari|              17|
|            Opera|              15|
|    Google Chrome|              12|
|        SeaMonkey|              11|
|  Android Browser|              11|
|          Firefox|              11|
|          Vivaldi|              11|
|   Microsoft Edge|              10|
|Internet Explorer|              10|
|       UC Browser|              10|
|     Apple Safari|               9|
|          Mozilla|               8|
|  Dolphin browser|               8|
|       Opera Neon|               8|
|       Opera Mini|               8|
+-----------------+----------------+



#### Payement Type

In [18]:
spark.sql("""select VisChip, count(VisChip) from Weblog group by VisChip""").show()

+----------------+--------------+
|         VisChip|count(VisChip)|
+----------------+--------------+
|        Discover|            49|
|            VISA|            62|
|American Express|            47|
|      MasterCard|            61|
+----------------+--------------+



#### Connection Type

In [19]:
spark.sql("""select ConxType, count(ConxType) from weblog group by ConxType""").show()

+--------+---------------+
|ConxType|count(ConxType)|
+--------+---------------+
|     LTE|             62|
|    CDMA|             52|
|     GSM|             46|
|   WiMax|             59|
+--------+---------------+

